# FT用データ生成スクリプト

In [1]:
# !conda install -y -c conda-forge kalpy \
# kaldi \
# pynini

# # パッケージインストール
# !pip install -r requirements.sbv.txt

In [2]:
# !pip list

In [3]:
# # mfa
# # 日本語辞書のダウンロード
# !mfa model download dictionary japanese_mfa

# # 日本語音響モデルのダウンロード
# !mfa model download acoustic japanese_mfa

Local version of model already exists (/users/s1f102201582/Documents/MFA/pretrained_models/dictionary/japanese_mfa.dict). Use the --ignore_cache flag to force redownloading.
Local version of model already exists (/users/s1f102201582/Documents/MFA/pretrained_models/acoustic/japanese_mfa.zip). Use the --ignore_cache flag to force redownloading.


## テキスト対話データ生成

In [4]:
import os
from typing import Literal
import ast

from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PDFMinerLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI


# .envファイル読み込み
load_dotenv("/users/s1f102201582/projects/mhcc-moshi/.env")

/nfs1/s1f102201582/anaconda3/envs/mfa_unit/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [5]:
#config
from os.path import join, expanduser

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
BASE_URL = "https://api.openai.iniad.org/api/v1"
MODEL='gemini-2.5-flash'
TEMPERATURE = 1.0
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

# 生成する音声のサンプリングレート
setting_sr = 16000

#対話音声データの個数を指定
gen_dial_num = 863

# すでに作成した対話データを削除するかどうか
IS_REMOVE_EXIST_FILE = False

version = "v3"

# ftに使うjsonとaudioの出力フォルダパス
home_dir = expanduser("~")
json_dir_path = join(home_dir, "projects/mhcc-moshi/moshi/data", version, "data_stereo")
audio_dir_path = join(home_dir, "projects/mhcc-moshi/moshi/data", version, "data_stereo")

# mfa関連のパス
model_dir = join(home_dir, "Documents/MFA/pretrained_models/acoustic/japanese_mfa.zip")
mfa_input_dir = join(home_dir, "projects/mhcc-moshi/moshi/data", version, "mfa_input")
mfa_output_dir = join(home_dir, "projects/mhcc-moshi/moshi/data", version, "mfa_output")


#RAGで読み取るPDFのパス
rag_pdf_dir = join(home_dir, "projects/mhcc-moshi/mental_docs/")

# SBVの音声合成モデルのパス
assets_root = join(home_dir, "projects/mhcc-moshi/moshi/model_assets")

In [6]:
base_paths = [
    json_dir_path,
    audio_dir_path,
    mfa_input_dir,
    mfa_output_dir,
]

for p in base_paths:
    if not os.path.isdir(p):
        os.makedirs(p)

In [7]:
# model定義
model = ChatGoogleGenerativeAI(
                 model=MODEL,
                 temperature=TEMPERATURE)

# 埋め込みモデル定義
embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=BASE_URL,
    model="text-embedding-3-large"
)

# データベース定義
vector_store = Chroma(
    collection_name="collection",
    embedding_function=embeddings,
    # persist_directory = "/path/to/db_file" # if necessary
)

In [8]:
loader = DirectoryLoader(
    rag_pdf_dir,
    glob="*.pdf",
    show_progress=True,
    loader_cls=PDFMinerLoader,
)
docs = loader.load()
print(f"Loaded {len(docs)} documents")

  0%|                                                                                             | 0/3 [00:00<?, ?it/s]Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cann

Loaded 3 documents


In [9]:
# Debug
# for doc in docs:
#     print("-------------------------------------------------")
#     print(doc.metadata)
#     print(len(doc.page_content))
#     print(doc.page_content[:100])

In [10]:
#読み込んだ文章データをオーバーラップ200文字で1000文字づつ分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True, # 分割前の文章のインデックスを追跡
)
splits = text_splitter.split_documents(docs)

# データベースにデータを追加
document_ids = vector_store.add_documents(documents=splits)

In [11]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query, k=2)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message

In [12]:
from typing import Literal

from pydantic import BaseModel, Field


class Dialogue(BaseModel):
    """対話データを構成する対話クラス"""
    speaker: Literal["A", "B"] = Field(..., description="話者。Aはカウンセラー、Bはクライエントを表す。")
    text: str = Field(..., description="話者が話した内容。")

class Dialogues(BaseModel):
    """カウンセリングを目的としたカウンセリング対話データ"""
    dialogues: list[Dialogue] = Field(..., description="対話データを構成する対話クラスのリスト。")

In [13]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model, 
    tools=[],
    middleware=[prompt_with_context],
    response_format=ToolStrategy(
        Dialogues,
        handle_errors="フォーマットに合うように、もう一度対話データを生成してください。"
    )
)

In [14]:
#promptを作成
import random

prompt_template = """あなたは、音声合成（TTS）用の対話シナリオを作成するプロのライターです。
以下の【要件】、【設定】、そして【指示】に従い、専門のカウンセラー (C) と クライアント (CL) のテキスト対話データを生成してください。

### 【目的】
生成するテキストは、音声合成エンジンで読み上げることを前提としています。そのため、台本のような流暢な書き言葉ではなく、人間同士が実際に会話する際の「話し言葉」を忠実に再現することを最優先とします。

### 【最重要要件：自然な会話の再現】
以下の要素を必ずテキストに含めてください。

1.  **極端に短い発話（ターン）の継続（最重要）:**
    * 一度の発話（セリフ）は、意図的に非常に短く区切ってください。
    * 人間同士の会話のように、短い言葉のキャッチボールが続くように構成してください。長い独白（モノローグ）や、一つの発話に多くの情報を詰め込むのは絶対に避けてください。

2.  **発話の途中の相槌（インターラプト）:**
    * これが最も重要です。**クライアント(CL)の発話**が少しでも長くなりそうな場合（例：10～15文字以上）、その**発話の途中に** **カウンセラー(C)**が「はい」「ええ」「うんうん」「なるほど」といった短い相槌を**独立した発話として**挟んでください。
    * クライアントは、その相槌を受けて話を続けるという、現実の会話で頻繁に起こる言葉の重なりや短いキャッチボールを忠実に再現してください。

3.  **クライアント(CL)の話し言葉の要素:**
    * 「よどみ（フィラー）」や「言い直し・ためらい」は、【指示】5で指定された内容に従って調整してください。

### 【設定】
* **話者:**
    * **C (カウンセラー):** 専門家。穏やかで、共感的・受容的なトーン。常に短い相槌を挟む。
    * **CL (クライアント):** 悩みを抱えている。

### 【指示】

1.  **指定された設定:**
    以下の指定された設定に基づいてシナリオを生成してください。
    * **クライアントのペルソナ:** {selected_persona}
    * **トピック:** {selected_topic}
    * **状況:** {selected_situation}
    * **クライアントの話し方:** {selected_style_name}

2.  **会話の開始ルール（名前の扱い）:**
    * シナリオは、必ず C (カウンセラー) がクライアントに呼びかける発話から始めてください。
    * 指定された【状況】が『初回』の場合: カウンセラーがクライアントの**名前（呼び方）を尋ねる発話**（例：「はじめまして。担当します〇〇です。まず、お名前（呼び方）を伺ってもよろしいですか？」）から始めてください。
    * 指定された【状況】が『初回』以外の場合: すでに名前は知っている設定とし、**名前を呼ばずに**（例：「こんにちは。その後の調子はいかがですか？」）始めてください。`〇〇さん`のようなプレースホルダーは**絶対に使用しないでください**。

3.  **会話の長さ:**
    * シナリオは、会話が**1分30秒〜2分程度**（テキスト量で約**8〜12往復程度**）続くようにしてください。

4.  **要件の遵守:**
    * 【最重要要件：自然な会話の再現】、特に「**1. 極端に短い発話**」と「**2. 発話の途中の相槌**」を厳密に守ってください。

5.  **クライアントの話し方の調整（指示）:**
    * 以下の指示に従って、**クライアント(CL)の発話**における話し方を調整してください。
    * **指示:** {selected_style_instruction}

6.  **ペルソナとトピックの統合（【ペルソナ】【トピック】変数に基づく）:**
    * 指定された【クライアントのペルソナ】（年齢・職業/立場）と【トピック】（悩み）を組み合わせて、クライアント(CL)の**具体的な会話内容や背景**を構築してください。
    * 例えば、ペルソナが「20代・会社員」でトピックが「職場の人間関係」なら、同僚や上司との関係に悩む若手社員の会話にしてください。
    * **重要:** クライアント自身が「私は〇〇です」のように、自分のペルソナを台詞で**明言しない**ようにしてください。あくまで会話の内容から**推測できる**ようにしてください。
"""

# トピックリスト（悩みの種類）(全40項目)
topic_list = [
  # 仕事・キャリア関連
  "仕事のプレッシャーや過労、バーンアウト",
  "職場の人間関係（上司、同僚、部下）",
  "キャリアプランの悩み、キャリアチェンジの不安",
  "転職・就職活動のストレス",
  "ハラスメント（パワハラ、モラハラなど）の影響",
  "仕事へのモチベーション低下、やる気が出ない",

  # 自己認識・感情関連
  "自己肯定感の低さ、自分を責めてしまう",
  "完璧主義、失敗への過度な恐れ",
  "他人の評価が過度に気になる、承認欲求",
  "劣等感（他人との比較）",
  "自分のやりたいことが分からない、アイデンティティの悩み",
  "感情のコントロールが難しい（怒り、イライラ、悲しみ）",
  "ネガティブ思考の癖、反芻思考（同じことをぐるぐる考えてしまう）",
  "焦燥感、何かに追われている感覚",
  "罪悪感（休むことへの罪悪感など）",
  "虚無感、むなしさ、生きがいを感じられない",
  "趣味や楽しみを感じられない（アンヘドニア）",
  
  # 対人関係
  "家族関係（親子、夫婦、兄弟、親戚）",
  "友人関係や恋愛関係の悩み",
  "コミュニケーションへの苦手意識（雑談、会議での発言など）",
  "人に頼ることができない、甘えられない",
  "他人の期待に応えすぎようとしてしまう（ピープルプリーザー）",
  "境界線（バウンダリー）の問題（NOと言えない）",
  "孤独感、疎外感",
  "HSP（繊細さ）に関する悩み",
  
  # 生活・健康関連
  "将来への漠然とした不安",
  "気分の落ち込み、無気力",
  "睡眠に関する悩み（寝付けない、途中で起きる、過眠）",
  "生活リズムの乱れ",
  "体調不良（頭痛、倦怠感、腹痛など）と気分の関連",
  "健康不安（病気への過度な心配）",

  # 習慣・行動関連
  "決断疲れ、何かを選ぶことができない",
  "先延ばし癖、物事が始められない",
  "SNS疲れ、デジタルデトックスの必要性",

  # 特定の出来事
  "ライフイベント（引っ越し、結婚、出産、育児、介護）に伴うストレス",
  "特定の出来事による軽いトラウマやフラッシュバック",
  "喪失体験（別れ、死別）からの回復（グリーフケア）",
  "過去の選択への後悔"
]

# 状況リスト（セッションの場面）
situation_list = [
  "初回",
  "初期",
  "中期（深掘り）",
  "中期（宿題の確認）",
  "中期（感情の表出）",
  "後期",
  "終了（終結）"
]

# クライアントのペルソナリスト（年齢と職業/立場のみ）
persona_list = [
    "20代・会社員",
    "30代・会社員",
    "40代・会社員",
    "50代・会社員",
    "20代・大学生",
    "20代・大学院生",
    "30代・管理職",
    "40代・主婦/主夫",
    "30代・フリーランス",
    "20代・アルバイト",
    "40代・パートタイム",
    "50代・自営業",
    "20代・求職中",
    "30代・育児休業中"
]

speaking_style_data = {
    "よどみが多い（ためらいがち）": "CLの発話に、フィラー（「えーと」「あのー」「なんていうか」「その…」）や「言い直し・言いよどみ」を意図的に【多く】挿入してください。言葉がすぐに出てこない、ためらいながら話す様子を強く表現してください。",
    "普通（自然な会話）": "【最重要要件】の指示通り、自然な会話の範囲で適度なフィラーと「言い直し」を挿入してください。",
    "スムーズ（よどみ少なめ）": "CLの発話におけるフィラーや「言い直し・言いよどみ」を意図的に【最小限に】し、比較的流暢に話すようにしてください。（ただし、「1. 極端に短い発話」のルールは守り続けてください）"
}

def gen_prompt_txt():
    # ランダムに選択
    selected_topic = random.choice(topic_list)
    selected_situation = random.choice(situation_list)
    selected_persona = random.choice(persona_list)
    selected_style_name = random.choice(list(speaking_style_data.keys()))
    selected_style_instruction = speaking_style_data[selected_style_name]

    # print("選ばれたトピック:", selected_topic)
    # print("選ばれた状況:", selected_situation)
    # print("選ばれたペルソナ:", selected_persona)
    # print("選ばれたクライアントのスタイル:", selected_style_name)

    # 変数をプロンプトに埋め込む
    final_prompt = prompt_template.format(
        selected_persona=selected_persona,
        selected_topic=selected_topic,
        selected_situation=selected_situation,
        selected_style_name=selected_style_name,
        selected_style_instruction=selected_style_instruction
    )
    
    return final_prompt

In [15]:
import time
from google.api_core.exceptions import ResourceExhausted

max_retries = 5
base_wait_time = 1 # minutes

# テキスト対話生成関数
def gen_txt_dialogue():
    prompt = gen_prompt_txt()

    # レート制限に引っかかることがあるため、例外処理
    for i in range(1, max_retries+1):
        try:
            resp = agent.invoke({"messages": [{"role": "user", "content": prompt}]})
            break
        except ResourceExhausted as e:
            if i < max_retries - 1:
                wait_time = (base_wait_time ** i) * 60
                time.sleep(wait_time)
            # max_retries回失敗した場合はエラーを起こす
            else:
                raise e
        except Exception as e:
            raise e

    dialogues_list = resp["structured_response"].dialogues
    return dialogues_list

In [16]:
#DEBUG
# txt_dialogue = gen_txt_dialogue()
# print(txt_dialogue)
# lst_dialogue = txt_to_lst(txt_dialogue)
# print(lst_dialogue)

## テキスト対話データを音声対話データに変換 

In [17]:
from style_bert_vits2.nlp import bert_models
from style_bert_vits2.constants import Languages
from pathlib import Path
from huggingface_hub import hf_hub_download
from style_bert_vits2.tts_model import TTSModel

bert_models.load_model(Languages.JP, "ku-nlp/deberta-v2-large-japanese-char-wwm")
bert_models.load_tokenizer(Languages.JP, "ku-nlp/deberta-v2-large-japanese-char-wwm")

# # 子春音あみ
# model_file = "koharune-ami/koharune-ami.safetensors"
# config_file = "koharune-ami/config.json"
# style_file = "koharune-ami/style_vectors.npy"
# hf_repo = "litagin/sbv2_koharune_ami"

# # あみたろ
# model_file = "amitaro/amitaro.safetensors"
# config_file = "amitaro/config.json"
# style_file = "amitaro/style_vectors.npy"
# hf_repo = "litagin/sbv2_amitaro"


# デフォルトの女性2
model_file = "jvnv-F2-jp/jvnv-F2_e166_s20000.safetensors"
config_file = "jvnv-F2-jp/config.json"
style_file = "jvnv-F2-jp/style_vectors.npy"
hf_repo = "litagin/style_bert_vits2_jvnv"

for file in [model_file, config_file, style_file]:
    print(file)
    hf_hub_download(hf_repo, file, local_dir=assets_root)

A_model = TTSModel(
    model_path=os.path.join(assets_root, model_file),
    config_path=os.path.join(assets_root, config_file),
    style_vec_path=os.path.join(assets_root, style_file),
    device="cuda",
)

# デフォルトの男性2
model_file = "jvnv-M2-jp/jvnv-M2-jp_e159_s17000.safetensors"
config_file = "jvnv-M2-jp/config.json"
style_file = "jvnv-M2-jp/style_vectors.npy"

for file in [model_file, config_file, style_file]:
    print(file)
    hf_hub_download(hf_repo, file, local_dir=assets_root)

B_model = TTSModel(
    model_path=os.path.join(assets_root, model_file),
    config_path=os.path.join(assets_root, config_file),
    style_vec_path=os.path.join(assets_root, style_file),
    device="cuda",
)

11-09 11:11:16 |  INFO  | bert_models.py:92 | Loaded the Languages.JP BERT model from ku-nlp/deberta-v2-large-japanese-char-wwm
11-09 11:11:17 |  INFO  | bert_models.py:154 | Loaded the Languages.JP BERT tokenizer from ku-nlp/deberta-v2-large-japanese-char-wwm
jvnv-F2-jp/jvnv-F2_e166_s20000.safetensors
jvnv-F2-jp/config.json
jvnv-F2-jp/style_vectors.npy
jvnv-M2-jp/jvnv-M2-jp_e159_s17000.safetensors
jvnv-M2-jp/config.json
jvnv-M2-jp/style_vectors.npy


In [18]:
def build_audio_synth_prompt(text_dialogue_list):
    resp = ""
    resp_header =  """あなたがこれから音声合成するテキストは以下の対話内容のワンフレーズです。
この対話の文脈に合うように音声合成してください。

<対話内容の全文>"""
    resp += resp_header
    for text_dial in text_dialogue_list:
        resp += f"\n{text_dial.speaker}: {text_dial.text}"
    return resp

In [19]:
from typing import Literal

def sbv_tts(text: str, speaker: Literal["A", "B"], assist_text=None):
    if speaker == "A":
        sr, audio = A_model.infer(
            text = text,
            style='Happy',
            style_weight=1,
            split_interval = 0.3,
            use_assist_text = True if assist_text is not None else None,
            assist_text = assist_text
        )
    else:
        sr, audio = B_model.infer(
            text = text,
            style='Sad',
            style_weight=1,
            split_interval = 0.3,
            use_assist_text = True if assist_text is not None else None,
            assist_text = assist_text
        )
    
    return sr, audio

In [20]:
import librosa
import numpy as np

def gen_audio_dialogue(text_dialogue_list, prompt):
    # 音声ファイルを順番に生成（ファイルは不要なのでwave配列で持つ）
    wav_data = []
    for dial in text_dialogue_list:
        speaker = dial.speaker
        sr, wav = sbv_tts(dial.text, speaker, prompt)

        # サンプリングレートを変換
        if sr != setting_sr:
            # 16ビット整数のデータを、-1.0から1.0の範囲に収まる浮動小数点数に正規化
            wav = wav.astype(np.float32) / 32768.0
            wav = librosa.resample(wav, orig_sr=sr, target_sr=setting_sr)

        # 0.3秒間の無音時間を追加
        duration_sec = 0.3
        num_silent_samples = int(setting_sr*duration_sec)
        silence = np.zeros(num_silent_samples, dtype=wav.dtype)
        wav_with_silence = np.concatenate((wav, silence))
        wav_data.append(wav_with_silence)
    
    # 最終的な音声長を決定
    max_len = sum([len(w) for w in wav_data])
    
    # ステレオ音声用（2チャンネル×最大長）の空配列をゼロ初期化で作成
    stereo = np.zeros((2, max_len), dtype=np.float32)
    
    pos = 0
    for i, wav in enumerate(wav_data):
        ch = i%2  # 0:左(A), 1:右(B)
        stereo[ch, pos:pos+len(wav)] += wav
        pos += len(wav)
    
    # 転置(-1,2)する
    stereo = stereo.T
    return stereo

## mfa(montreal force alignment)による音声アラインメント

In [21]:
import copy

def correct_json(full_text, align_json):
    new_align_json = copy.deepcopy(align_json)
    segments = new_align_json["tiers"]["words"]["entries"]
    checked_len = 0
    prev_checked_len = 0
    i = 0
    while i < len(segments):
        if re.search(f"^<unk>|<sil>$", segments[i][2]):
            if i == 0:
                if re.search(f"^<unk>|<sil>$", segments[i+1][2]):
                    end_time = 0
                    while re.search(f"^<unk>|<sil>$", segments[i+1][2]):
                        end_time = segments[i+1][1]
                        segments.pop(i+1)
                    segments[i][1] = end_time
                
                m = re.search(f"^(.+?){segments[i+1][2]}", full_text[checked_len:])
                match_text = m.groups()
                segments[i][2] = match_text[0]
            elif i == len(segments)-1:
                m = re.search(f"{segments[i-1][2]}(.+?)$", full_text[checked_len:])
                match_text = m.groups()
                segments[i][2] = match_text[0]
            else:
                if re.search(f"^<unk>|<sil>$", segments[i+1][2]):
                    end_time = 0
                    while re.search(f"^<unk>|<sil>$", segments[i+1][2]):
                        end_time = segments[i+1][1]
                        segments.pop(i+1)
                    segments[i][1] = end_time
                m = re.search(f"^{segments[i-1][2]}(.+?){segments[i+1][2]}", full_text[prev_checked_len:])
                match_text = m.groups()
                segments[i][2] = match_text[0]
        else:
            if re.search(f"^([。、,.!?！？…「」]){segments[i][2]}.*$", full_text[checked_len:]):
                m = re.search(f"^([。、,.!?！？…「」]){segments[i][2]}.*$", full_text[checked_len:])
                match_punc = m.groups()
                segments[i][2] = match_punc[0] + segments[i][2]
            elif re.search(f"^{segments[i][2]}([。、,.!?！？…「」]).*$", full_text[checked_len:]):
                m = re.search(f"^{segments[i][2]}([。、,.!?！？…「」]).*$", full_text[checked_len:])
                match_punc = m.groups()
                segments[i][2] = segments[i][2] + match_punc[0]
                
        prev_checked_len = checked_len
        checked_len += len(segments[i][2])
        i += 1
    return new_align_json

In [22]:
from os.path import join, expanduser
import subprocess
import json
import re
import shutil

def alignment_channel(channel, target_dir_name):
    input_dir_path = join(mfa_input_dir, target_dir_name)
    output_dir_path = join(mfa_output_dir, target_dir_name)
    os.makedirs(input_dir_path, exist_ok=True)
    os.makedirs(output_dir_path, exist_ok=True)

    subprocess.run([
        "mfa",
        "align",
        input_dir_path,
        "japanese_mfa",
        model_dir,
        output_dir_path,
        "--quiet",
        "--overwrite",
        "--clean",
        "--final_clean",
        "--output_format", "json",
        "--beam", "1000",
        "--retry_beam", "4000",
    ])      

def parse_ft_json(json_data):
    result = {"alignments": []}

    segments = json_data["tiers"]["words"]["entries"]
    for segment in segments:
        result["alignments"].append([
            segment[2],
            [segment[0], segment[1]],
            "SPEAKER_MAIN"
        ])
    result["alignments"].sort(key=lambda x: x[1][0])
    return result

def alignment_audio_dialogue(text_dialogue_list, audio_path, idx):
    json_list = []
    audio, sr = sf.read(audio_path)
    
    result = ""
    target_dir_name = str(idx)
    target_dir = os.path.join(mfa_input_dir, target_dir_name)
    if not os.path.isdir(target_dir):
        os.makedirs(target_dir)

    target_text_file = os.path.join(target_dir, f"{idx}.txt")

    oneline_text = ""
    for dial in text_dialogue_list:
        result += dial.text + "\n"
        oneline_text += dial.text
    with open(target_text_file, "w") as f:
        f.write(result)

    wav_name = f"{idx}.wav"
    src_wav_path = os.path.join(audio_dir_path, wav_name)
    dist_wav_path = os.path.join(target_dir, wav_name)
    shutil.copy(src_wav_path, dist_wav_path)

    alignment_channel(audio, target_dir_name)
    json_path = os.path.join(mfa_output_dir, target_dir_name, f"{idx}.json")
    json_data = ""
    with open(json_path, "r") as f:
        json_data = json.load(f)

    try:
        correct_json_data = correct_json(oneline_text, json_data)
        ft_json = parse_ft_json(correct_json_data)
    except:
        print(f"jsonファイル {idx}.json の訂正に失敗しました。")
        ft_json= parse_ft_json(json_data)
    
    return ft_json

## フォルダ初期化

In [23]:
import re

def get_file_name():
    wav_file_pattern = r"^(\d+)\.wav$"
    num = -1
    for file in os.listdir(audio_dir_path):
        if not os.path.exists(os.path.join(audio_dir_path, file)):
            continue
        if not re.match(wav_file_pattern, file):
            continue
    
        match_obj = re.match(wav_file_pattern, file)
        get_number = int(match_obj.groups()[0])
    
        if num < get_number:
            num = get_number
    return num

In [24]:
from glob import glob
import shutil

def delete_files(dir_path):
    shutil.rmtree(dir_path)
    os.makedirs(dir_path)

if IS_REMOVE_EXIST_FILE:
    file_name_num = -1
    for dir_path in base_paths:
        delete_files(dir_path)
else:
    file_name_num = get_file_name()

## メイン処理

In [25]:
%%time

import soundfile as sf
import json

for i in range(file_name_num+1, gen_dial_num+file_name_num+1):

    # テキスト生成
    txt_dialogue_list = gen_txt_dialogue()

    # 音声合成のためのプロンプト生成
    audio_synth_prompt = build_audio_synth_prompt(txt_dialogue_list)

    # 対話テキストを音声合成
    stereo = gen_audio_dialogue(txt_dialogue_list, audio_synth_prompt)
    
    wav_name = f"{i}.wav"
    audio_file_path = os.path.join(audio_dir_path, wav_name)

    # wavファイル出力
    sf.write(audio_file_path, stereo, setting_sr)

    # 音声アラインメント
    json_data = alignment_audio_dialogue(txt_dialogue_list, audio_file_path, i)

    json_name = f"{i}.json"
    json_file_path = os.path.join(json_dir_path, json_name)
    
    # JSON出力
    with open(json_file_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=2)

11-09 11:11:34 |  INFO  | tts_model.py:259 | Start generating audio data from text:
はじめまして。担当しますカウンセラーです。まず、お名前（呼び方）を伺ってもよろしいですか？
11-09 11:11:34 |  INFO  | infer.py:24 | Using JP-Extra model


/nfs1/s1f102201582/anaconda3/envs/mfa_unit/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


11-09 11:11:39 |  INFO  | safetensors.py:50 | Loaded '/users/s1f102201582/projects/mhcc-moshi/moshi/model_assets/jvnv-F2-jp/jvnv-F2_e166_s20000.safetensors' (iteration 166)


/nfs1/s1f102201582/anaconda3/envs/mfa_unit/lib/python3.12/site-packages/pyopenjtalk/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


11-09 11:11:48 |  INFO  | tts_model.py:324 | Audio data generated successfully
11-09 11:11:51 |  INFO  | tts_model.py:259 | Start generating audio data from text:
はい、〇〇と申します。今日は、決めることが多くて、すごく疲れてしまっていて…
11-09 11:11:51 |  INFO  | infer.py:24 | Using JP-Extra model


/nfs1/s1f102201582/anaconda3/envs/mfa_unit/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


11-09 11:11:54 |  INFO  | safetensors.py:50 | Loaded '/users/s1f102201582/projects/mhcc-moshi/moshi/model_assets/jvnv-M2-jp/jvnv-M2-jp_e159_s17000.safetensors' (iteration 159)
11-09 11:11:55 |  INFO  | tts_model.py:324 | Audio data generated successfully
11-09 11:11:55 |  INFO  | tts_model.py:259 | Start generating audio data from text:
はい
11-09 11:11:55 |  INFO  | tts_model.py:324 | Audio data generated successfully
11-09 11:11:55 |  INFO  | tts_model.py:259 | Start generating audio data from text:
その、特に育児休業中なんですけど、毎日小さなことでも、色々決めなきゃいけないことが多くて
11-09 11:11:56 |  INFO  | tts_model.py:324 | Audio data generated successfully
11-09 11:11:56 |  INFO  | tts_model.py:259 | Start generating audio data from text:
うんうん
11-09 11:11:56 |  INFO  | tts_model.py:324 | Audio data generated successfully
11-09 11:11:56 |  INFO  | tts_model.py:259 | Start generating audio data from text:
それで、もう何を選んだらいいのか、わからなくなっちゃうんです
11-09 11:11:56 |  INFO  | tts_model.py:324 | Audio data generated successfully
11-09 11:

 INFO     Setting up corpus information...                                      
 INFO     Loading corpus from source files...                                   
 INFO     Found 1 speaker across 1 file, average number of utterances per       
          speaker: 1.0                                                          
 INFO     Initializing multiprocessing jobs...                                  
 WARNING  Number of jobs was specified as 3, but due to only having 1 speakers, 
          MFA will only use 1 jobs. Use the --single_speaker flag if you would  
          like to split utterances across jobs regardless of their speaker.     
 INFO     Normalizing text...                                                   
 INFO     Generating MFCCs...                                                   
 INFO     Calculating CMVN...                                                   
 INFO     Generating final features...                                          
 INFO     Creating corpus sp

CPU times: user 1min 6s, sys: 4.21 s, total: 1min 10s
Wall time: 3min 45s
